In [1]:
import pandas as pd
import numpy as np
import ast

In [5]:
#Se crea la variable "peliculas", se la transforma en un DataFrame de Pandas y se la carga con los CSV pedidos

peliculas = pd.DataFrame
peliculas = pd.read_csv("C:/Users/Fanny/Documents/Henry/Labs/movies_dataset.csv")

C:\Users\fanny\AppData\Local\Temp\ipykernel_1440\2112098861.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  peliculas = pd.read_csv("C:/Users/Fanny/Documents/Henry/Labs/movies_dataset.csv")


In [9]:
#Se crea la variable "creditos", se la transforma en un DataFrame de Pandas y se la carga con los CSV pedidos que se encuentran en los campos "cast" y "crew"

creditos = pd.read_csv("C:/Users/Fanny/Documents/Henry/Labs/credits.csv", usecols=['cast', 'crew'])

In [ ]:
# Se agregan las columnas 'cast' y 'crew' del DataFrame "creditos" al DataFrame "peliculas" para unificar todos los datos necesarios

peliculas['cast'] = creditos['cast']
peliculas['crew'] = creditos['crew']


In [ ]:
#Creo la función "extraer_nombre_actor" que va a dejar en el DataFrame solo los nombres de los actores

def extraer_nombre_actor(cast):
    if pd.isnull(cast):
        return np.nan
    else:
        try:
            actor_list = ast.literal_eval(cast)
            actor_names = [actor['name'] for actor in actor_list]
            return ', '.join(actor_names)  
        except (ValueError, TypeError):
            return np.nan

In [ ]:
# Aplico la función "extraer_nombre_actor" a la columna 'cast'

peliculas['actores'] = peliculas['cast'].apply(extraer_nombre_actor)

In [ ]:
#Creo la función "extraer_director" que va a dejar en el DataFrame solo el nombre del director en la columna "crew"

def extraer_director(crew):
    if pd.isnull(crew):
        return np.nan
    else:
        try:
            crew_list = ast.literal_eval(crew)
            for member in crew_list:
                if member['job'] == 'Director':
                    return member['name']
            return np.nan
        except (ValueError, TypeError):
            return np.nan

In [ ]:
# Aplico la función "extraer_director" a la columna 'crew' y creo la columna 'director'

peliculas['director'] = peliculas['crew'].apply(extraer_director)

In [ ]:
#Elimino la columna "crew"

peliculas.drop('crew', axis=1, inplace=True)

In [ ]:
#Creo la funcion "extraer_nombre_franquicia"  para desanidar la columna 'belongs_to_collection' y extraer el nombre de la franquicia a la que corresponde cada pelicula

def extraer_nombre_franquicia(collection):
    if pd.isnull(collection):
        return np.nan
    else:
        try:
            return ast.literal_eval(collection)['name']
        except (ValueError, TypeError):
            return np.nan


In [ ]:
# Luego aplico la funcion a 'belongs_to_collection'

peliculas['belongs_to_collection'] = peliculas['belongs_to_collection'].apply(extraer_nombre_franquicia)

In [ ]:
# Creo la función "extraer_lenguaje" para desanidar la columna 'spoken_languages' y extraer el idioma de cada pelicula

def extraer_lenguaje(languages):
    if pd.isnull(languages):
        return pd.NA
    try:
        languages_list = eval(languages)
        language_names = [lang['name'] for lang in languages_list]
        return ", ".join(language_names)
    except (ValueError, TypeError):
        return pd.NA

In [ ]:
# Aplico la función para desanidar 'spoken_languages' y obtener el idioma

peliculas['spoken_languages'] = peliculas['spoken_languages'].apply(extraer_lenguaje)

In [ ]:
# Creo la función "extraer_genero" para desanidar la columna 'genres' y extraer el género de cada pelicula

def extraer_genero(genres):
    if pd.isnull(genres):
        return pd.NA
    try:
        genres_list = eval(genres)
        genre_names = [genre['name'] for genre in genres_list]
        return ", ".join(genre_names)
    except (ValueError, TypeError):
        return pd.NA

In [ ]:
# Aplico la función "extraer_genero" para desanidar 'genres' y obtener los generos de cada película

peliculas['genres'] = peliculas['genres'].apply(extraer_genero)

In [ ]:
#Creo la función "extraer_productoras" para desanidar la columna 'production_companies' y extraer los nombres de las compañías de producción

def extraer_productoras(companies):
    if pd.isnull(companies):
        return pd.NA
    try:
        companies_list = eval(companies)
        company_names = [company['name'] for company in companies_list]
        return ", ".join(company_names)
    except (ValueError, TypeError):
        return pd.NA

In [ ]:
# Aplico la función para desanidar 'production_companies' y obtener los nombres de las compañías productoras

peliculas['production_companies'] = peliculas['production_companies'].apply(extraer_productoras)

In [ ]:
# Creo la función "extaer_pais" para desanidar la columna 'production_countries' y extraer solamente los países de producción

def extaer_pais(countries):
    if pd.isnull(countries):
        return pd.NA
    try:
        countries_list = eval(countries)
        country_names = [country['name'] for country in countries_list]
        return ", ".join(country_names)
    except (ValueError, TypeError):
        return pd.NA

In [ ]:
# Aplico la función "extaer_pais" para desanidar 'production_countries' y obtener los nombres de los países de producción

peliculas['production_countries'] = peliculas['production_countries'].apply(extaer_pais)

In [ ]:
# Relleno los valores nulos en la columna "budget" con 0

peliculas['budget'].fillna(0, inplace=True)

In [ ]:
# Relleno los valores nulos en la columna "revenue" con 0

peliculas['revenue'].fillna(0, inplace=True)

In [ ]:
# Elimino los valores nulos del campo 'release_date'

peliculas.dropna(subset=['release_date'], inplace=True)

In [ ]:
# Convierto la columna 'release_date' a formato de fecha y manejo los valores no válidos

peliculas['release_date'] = pd.to_datetime(peliculas['release_date'], errors='coerce')

In [ ]:
# Elimino las filas que contienen valores NaT (no válidos)

peliculas.dropna(subset=['release_date'], inplace=True)

In [ ]:
# Doy formato final a las fechas 'AAAA-mm-dd'

peliculas['release_date'] = peliculas['release_date'].dt.strftime('%Y-%m-%d')

In [ ]:
# Creo la función para calcular el retorno de inversión

def calculate_return(row):
    revenue = pd.to_numeric(row['revenue'], errors='coerce')
    budget = pd.to_numeric(row['budget'], errors='coerce')
    if pd.notna(revenue) and pd.notna(budget) and budget != 0:
        return revenue / budget
    return 0

In [ ]:
# Creo la nueva columna 'return' aplicando la función "calculate_return"

peliculas['return'] = peliculas.apply(calculate_return, axis=1)

In [ ]:
# Creo la variable "columnas_descartadaas" y la completo con las columnas a elimanr del dataframe
columnas_descartadaas = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'id', 'status', 'tagline']

# Elimino las columnas
peliculas.drop(columns=columnas_descartadaas, inplace=True)